In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
df = pd.read_csv("../Data/Processed/cleaned_merged.csv")

In [41]:

# Base daily aggregate metrics with mean, total, std

timely_data = df.groupby('date').agg(
    cpu_mean=('usage_cpu', 'mean'),
    cpu_tot=('usage_cpu', 'sum'),
    cpu_std=('usage_cpu', 'std'),
    
    storage_mean=('usage_storage', 'mean'),
    storage_tot=('usage_storage', 'sum'),
    storage_std=('usage_storage', 'std'),
    
    users_mean=('users_active', 'mean'),
    users_tot=('users_active', 'sum'),
    users_std=('users_active', 'std'),
    
    economic_index=('economic_index', 'first'),
    cloud_market_demand=('cloud_market_demand', 'first'),
    holiday=('holiday', 'first'),
    
    unique_regions=('region', 'nunique'),
    total_records=('resource_type', 'count')
).reset_index()

# Round to 2 decimals
timely_data = timely_data.round(2)


# Helper to collect extremes in JSON

def get_extremes(sub_df, col):
    min_row = sub_df.loc[sub_df[col].idxmin()]
    max_row = sub_df.loc[sub_df[col].idxmax()]
    return {
        "min": {
            "value": round(float(min_row[col]), 2),
            "region": min_row["region"],
            "resource": min_row["resource_type"]
        },
        "max": {
            "value": round(float(max_row[col]), 2),
            "region": max_row["region"],
            "resource": max_row["resource_type"]
        }
    }

# Build JSON column per date
extremes_info = []
for d, sub in df.groupby("date"):
    info = {
        "date": d,
        "cpu": get_extremes(sub, "usage_cpu"),
        "storage": get_extremes(sub, "usage_storage"),
        "users": get_extremes(sub, "users_active")
    }
    extremes_info.append(info)

extremes_df = pd.DataFrame(extremes_info)


# Active regions list
region_data = df.groupby('date')['region'].unique().reset_index()
region_data.rename(columns={'region': 'active_regions'}, inplace=True)

# Resources per region mapping
resources_data = (
    df.groupby(['date', 'region'])['resource_type']
    .unique()
    .reset_index()
    .groupby('date')
    .apply(lambda x: {row['region']: list(row['resource_type']) for _, row in x.iterrows()})
    .reset_index(name='resources_per_region')
)


# Merge everything together
final_data = (
    timely_data
    .merge(extremes_df, on="date")
    .merge(region_data, on='date')
    .merge(resources_data, on='date')
)

# Ensure all numeric columns are rounded
numeric_cols = final_data.select_dtypes(include=[np.number]).columns
final_data[numeric_cols] = final_data[numeric_cols].round(2)

final_data



C:\Users\sujay\AppData\Local\Temp\ipykernel_33536\1851898177.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: {row['region']: list(row['resource_type']) for _, row in x.iterrows()})


,date,cpu_mean,cpu_tot,cpu_std,storage_mean,storage_tot,storage_std,users_mean,users_tot,users_std,economic_index,cloud_market_demand,holiday,unique_regions,total_records,cpu,storage,users,active_regions,resources_per_region
0,2023-01-01,74.92,899,14.44,1509.00,18108,426.24,387.00,4644,93.94,104.97,0.99,1,4,12,"{'min': {'value': 51.0, 'region': 'north europ...","{'min': {'value': 621.0, 'region': 'east us', ...","{'min': {'value': 221.0, 'region': 'north euro...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
1,2023-01-02,75.17,902,20.58,1286.83,15442,434.09,318.33,3820,98.52,106.48,1.15,0,4,12,"{'min': {'value': 51.0, 'region': 'north europ...","{'min': {'value': 534.0, 'region': 'north euro...","{'min': {'value': 201.0, 'region': 'southeast ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
2,2023-01-03,76.50,918,12.20,1147.75,13773,474.97,373.58,4483,82.26,97.66,0.98,0,4,12,"{'min': {'value': 57.0, 'region': 'east us', '...","{'min': {'value': 564.0, 'region': 'north euro...","{'min': {'value': 208.0, 'region': 'north euro...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
3,2023-01-04,72.08,865,16.55,1126.92,13523,295.51,327.17,3926,93.44,115.79,1.08,0,4,12,"{'min': {'value': 50.0, 'region': 'north europ...","{'min': {'value': 634.0, 'region': 'east us', ...","{'min': {'value': 214.0, 'region': 'southeast ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
4,2023-01-05,76.50,918,14.43,1580.50,18966,357.88,370.08,4441,78.45,95.31,1.05,0,4,12,"{'min': {'value': 56.0, 'region': 'west us', '...","{'min': {'value': 984.0, 'region': 'east us', ...","{'min': {'value': 228.0, 'region': 'east us', ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,2023-03-27,78.00,936,14.61,1392.75,16713,446.87,326.08,3913,105.17,91.10,0.92,0,4,12,"{'min': {'value': 53.0, 'region': 'southeast a...","{'min': {'value': 842.0, 'region': 'north euro...","{'min': {'value': 207.0, 'region': 'west us', ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
86,2023-03-28,70.92,851,15.55,1379.50,16554,449.83,389.08,4669,74.68,99.23,1.03,0,4,12,"{'min': {'value': 53.0, 'region': 'east us', '...","{'min': {'value': 548.0, 'region': 'southeast ...","{'min': {'value': 275.0, 'region': 'east us', ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
87,2023-03-29,82.33,988,15.88,1051.58,12619,412.49,316.17,3794,77.41,102.77,1.08,0,4,12,"{'min': {'value': 57.0, 'region': 'southeast a...","{'min': {'value': 500.0, 'region': 'southeast ...","{'min': {'value': 216.0, 'region': 'west us', ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."
88,2023-03-30,78.92,947,17.40,1120.67,13448,421.33,377.75,4533,60.97,100.13,1.15,0,4,12,"{'min': {'value': 51.0, 'region': 'north europ...","{'min': {'value': 540.0, 'region': 'southeast ...","{'min': {'value': 283.0, 'region': 'southeast ...","[east us, west us, north europe, southeast asia]","{'east us': ['vm', 'storage', 'container'], 'n..."


In [38]:
df["storage_allocated"] = df.groupby(["region", "resource_type"])["usage_storage"].transform("max")
df["storage_efficiency"] = (df["usage_storage"]/df["storage_allocated"]*100).round(2)
df




,date,region,resource_type,usage_cpu,usage_storage,users_active,economic_index,cloud_market_demand,holiday,storage_allocated,storage_efficiency
0,2023-01-01,east us,vm,88,1959,470,104.97,0.99,1,1992,98.34
1,2023-01-01,east us,storage,92,1595,388,104.97,0.99,1,1992,80.07
2,2023-01-01,east us,container,70,621,414,104.97,0.99,1,1995,31.13
3,2023-01-01,west us,vm,60,1982,287,104.97,0.99,1,1986,99.80
4,2023-01-01,west us,storage,85,1371,351,104.97,0.99,1,1966,69.74
...,...,...,...,...,...,...,...,...,...,...,...
1075,2023-03-31,north europe,storage,84,734,412,97.35,1.27,0,1964,37.37
1076,2023-03-31,north europe,container,98,1650,422,97.35,1.27,0,1976,83.50
1077,2023-03-31,southeast asia,vm,74,1802,493,97.35,1.27,0,1984,90.83
1078,2023-03-31,southeast asia,storage,77,776,306,97.35,1.27,0,1949,39.82


In [40]:

final_data.to_csv("../Data/Processed/feature_engineered.csv",index=False)

df.to_csv("../Data/Processed/insights.csv",index=False)